# ProjectQ Example
## Compiling code for AWS Braket Service

AWS Braket devices supported: Simulator 'SV1', Rigetti 'Aspen-8' and IonQ device 'IonQ'

You need to have a valid AWS account and created a pair of access key/secret key, and have activated the braket service, therefore a specific S3 bucket and folder associated to the service should be configured.

First the usual imports

In [ ]:
from projectq import MainEngine
from projectq.backends import AWSBraketBackend
from projectq.ops import Measure, H, C, X, All


Configure the credentials, the S3 folder and the device to be used

In [ ]:
creds = ['AWS_ACCESS_KEY', 'AWS_SECRET_KEY'] # replace with your Access key and Secret key

s3_folder = ['S3Bucket', 'S3Directory'] # replace with your S3 bucket and directory

device = 'SV1' # replace by the device you want to use

Configure the engine

In [ ]:

eng = MainEngine(AWSBraketBackend(use_hardware=False, credentials=creds, s3_folder=s3_folder, num_runs=10, interval=10),engine_list=[])

Create the circuit

In [ ]:
    qureg = eng.allocate_qureg(3)
    # entangle the qureg
    H | qureg[0]
    H | qureg[1]
    C(X) | (qureg[1], qureg[2])
    C(X) | (qureg[0], qureg[1])
    H | qureg[0]
    C(X) | (qureg[1], qureg[2])
    # measure; should be all-0 or all-1
    All(Measure) | qureg
    # run the circuit
    eng.flush()


Get the probabilities

In [ ]:
prob_dict = eng.backend.get_probabilities(qureg)
print("Probabilites for each of the results: ", prob_dict)

We can retrieve the result later on (of this job or a previously executed one) using the Task Arn provided when it was run. You have to remember the ammount of qubits involved in the job and the order you used.

In [ ]:

task_arn = 'your_task_arn' # replace with the actual TaskArn you wnat to use
eng1 = MainEngine(AWSBraketBackend(retrieve_execution=task_arn, credentials=creds, num_retries=2, verbose=True), engine_list=[])

qureg1 = eng1.allocate_qureg(3)
eng1.flush()

prob_dict1 = eng1.backend.get_probabilities(qureg1)
print("Probabilities ", prob_dict1)


Plot the histogram with the probabilities

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from projectq.libs.hist import histogram

histogram(eng1.backend, qureg1)
plt.show()